## Multi Scale CNN Cascade Network

This notebook implements the multi scale cnn cascasde network proposed in the paper WIDER FACES

In [1]:
import os
import requests
import zipfile
import shutil
from glob import iglob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

In [2]:
def download_file_from_google_drive(id, destination):
    
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [3]:
# define paths
current_path = os.getcwd()
data_path = os.path.join(current_path, "data")

# make data directory
try:
    os.makedirs(data_path)
except Exception as e:
    pass

In [4]:
# download wider face training data
if not os.path.exists(os.path.join(data_path, "train.zip")):
    print("downloading ... train.zip -- 1.47 GB")
    download_file_from_google_drive(
        "0B6eKvaijfFUDQUUwd21EckhUbWs", 
        os.path.join(data_path,"train.zip"))

downloading ... train.zip -- 1.47 GB


In [5]:
# download wider face validation data
if not os.path.exists(os.path.join(data_path,"val.zip")):
    print("downloading ... val.zip -- 362.8 MB")
    download_file_from_google_drive(
        "0B6eKvaijfFUDd3dIRmpvSk8tLUk", 
        os.path.join(data_path,"val.zip"))


downloading ... val.zip -- 362.8 MB


In [6]:
# download annotations
url = 'http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip'
if not os.path.exists(os.path.join(data_path, 'wider_face_split.zip')):
    print("downloading ... wider_face_split.zip -- 3.6 MB")
    r = requests.get(url, allow_redirects = True)
    open(os.path.join(data_path, 'wider_face_split.zip'), 'wb').write(r.content)

downloading ... wider_face_split.zip -- 3.6 MB


In [7]:
# unzip training data
if not os.path.exists(os.path.join(data_path,"WIDER_train")):
    with zipfile.ZipFile(os.path.join(data_path,"train.zip"),"r") as zip_ref:
        zip_ref.extractall(data_path)

In [8]:
# unzip validation data
if not os.path.exists(os.path.join(data_path,"WIDER_val")):
    with zipfile.ZipFile(os.path.join(data_path,"val.zip"),"r") as zip_ref:
        zip_ref.extractall(data_path)

In [9]:
# unzip annotations
if not os.path.exists(os.path.join(data_path,"wider_face_split")):
    with zipfile.ZipFile(os.path.join(data_path,"wider_face_split.zip"),"r") as zip_ref:
        zip_ref.extractall(data_path)

In [13]:
# remove zip files
if os.path.exists(os.path.join(data_path, "wider_face_split.zip")):
    os.remove(os.path.join(data_path, "wider_face_split.zip"))
if os.path.exists(os.path.join(data_path, "train.zip")):
    os.remove(os.path.join(data_path, "train.zip"))
if os.path.exists(os.path.join(data_path, "val.zip")):
    os.remove(os.path.join(data_path, "val.zip"))

In [14]:
path = os.path.join(data_path, "wider_face_split/wider_face_train_bbx_gt.txt")

In [70]:
def get_training_data(path, size):
    # open file and read each line
    f = open(path, 'r')
    lines = f.readlines()
    # create empty array for training data
    training_data = []
    # iterate over lines
    i = 0
    while i < len(lines):
        # return if enough samples have been found
        if len(training_data) >= size:
            return training_data
        # initialize new picture
        picture = {}
        # picture must start with file path
        assert lines[i].endswith(".jpg\n"), "read fault " + lines[i]
        picture['path'] = lines[i][:-1] # remove \n character
        # next line contains number of faces
        i += 1
        number_of_faces = int(lines[i])
        i += 1
        faces = []
        for j in range(i, i + number_of_faces):
            face = {}
            features = lines[j]
            features = features.split(' ')
            bb = features[:4]
            face['bb'] = [int(i) for i in bb]
            face['blur'] = features[4]
            face['expression'] = features[5]
            face['illumination'] = features[6]
            face['occlusion'] = features[7]
            face['pose'] = features[8]
            face['invalid'] = features[9]
            faces.append(face)

        # increase the counter
        i += number_of_faces
        # add picture to training set
        picture['faces'] = faces
        training_data.append(picture)

    return training_data

In [71]:
train_data = get_training_data(path, 3)

In [79]:
len(train_data[2]['faces'])

21

In [41]:
f = open(path, 'r')
lines = f.readlines()
filenames = []
for i in range(0, 10):
    if lines[i].endswith(".jpg\n"):
        filenames.append(lines[i])
        print(lines[i][:-1])

0--Parade/0_Parade_marchingband_1_849.jpg
0--Parade/0_Parade_Parade_0_904.jpg
0--Parade/0_Parade_marchingband_1_799.jpg
